In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install pandas numpy scikit-learn yfinance matplotlib

Looking in indexes: https://download.pytorch.org/whl/cu121


# PTCNet Training & Visualization
This notebook trains a PTCNet model on financial time-series using the existing data pipeline.
It includes:
- Data loading with `variable_length_collate`
- Visualization of time-series and correlations
- Supervised training loop (BCEWithLogitsLoss)
- Validation and Testing

In [3]:
# --- COLAB / REMOTE SETUP ---
import os
import sys

# Check if we are in Colab
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("Detected Google Colab environment.")
    
    # 1. Mount Drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
    
    # 2. Define your project path
    # ⚠️ CHANGE THIS to match your folder structure in Google Drive
    # e.g. "Projects/MAIA/FinancialTSC"
    PROJECT_PATH_DRIVE = "/content/drive/MyDrive/Projects/MAIA/FinancialTSC"
    
    if os.path.exists(PROJECT_PATH_DRIVE):
        os.chdir(PROJECT_PATH_DRIVE)
        print(f"Changed working directory to: {os.getcwd()}")
    else:
        print(f"Path not found: {PROJECT_PATH_DRIVE}")
        print("Please create this folder in Drive and upload your 'src' and 'data' folders there.")
        print("Or update PROJECT_PATH_DRIVE in this cell.")

# Add current directory to sys.path
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# Verify structure
if os.path.exists("src"):
    print("✅ 'src' package found.")
else:
    print("❌ 'src' package NOT found in current directory.")
    print(f"Current contents: {os.listdir()}")

🚀 Detected Google Colab environment.


ValueError: mount failed

In [2]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Ensure src is in path
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

from src.data.data_lauder import prepare_and_save_datasets, get_dataloaders
from src.ptck_arch.arch import PTCNet, ModelA_NoInteractions, ModelB_WithInteractions
from src.plot.data_plot import plot_batch_by_label

# Setup Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"Current working directory: {os.getcwd()}")

ModuleNotFoundError: No module named 'src'

In [ ]:
# --- Data Loading ---
# Ensure datasets exist
if not os.path.exists("data/train_dataset.pth"):
    print("Datasets not found. Generating...")
    prepare_and_save_datasets()

# Load DataLoaders
batch_size = 64
train_loader, val_loader, test_loader, input_dim = get_dataloaders(
    data_dir="data",
    batch_size=batch_size,
    num_workers=0
)

print(f"Input Features: {input_dim}")
print(f"Train samples: {len(train_loader.dataset)}")
print(f"Val samples:   {len(val_loader.dataset)}")
print(f"Test samples:  {len(test_loader.dataset)}")

# Get one batch for visualization
# Note: variable_length_collate returns (X, y, time_mask, feat_mask)
# We unpack all 4 but only use X and y for now.
batch = next(iter(train_loader))
X_batch, y_batch = batch[0], batch[1]

print(f"X_batch shape: {X_batch.shape} (B, F, T)")
print(f"y_batch shape: {y_batch.shape} (B, 1)")

In [ ]:
# --- Visualization (Temporal) ---
print("Visualizing Class 0 vs Class 1 (Temporal Mode)...")
plot_batch_by_label(
    X_batch, 
    y_batch, 
    max_samples_per_class=3, 
    n_features_to_plot=5, 
    mode="temporal"
)

In [ ]:
# --- Visualization (Correlation) ---
print("Visualizing Class 0 vs Class 1 (Correlation Mode)...")
plot_batch_by_label(
    X_batch, 
    y_batch, 
    max_samples_per_class=10, 
    n_features_to_plot=input_dim, 
    mode="corr"
)

In [ ]:
# --- Model Instantiation ---

# Choose your model: "PTCNet", "ModelA", or "ModelB"
MODEL_TYPE = "ModelB" 

print(f"Initializing {MODEL_TYPE}...")

common_params = dict(
    n_features=input_dim,
    n_blocks=10,
    kernel_size=3,
    dropout=0.1,
    pool_type="none",
    pool_every=1,
    n_domains=None,
    mlp_hidden_dim=256,
    mlp_blocks=4,
)

if MODEL_TYPE == "ModelA":
    model = ModelA_NoInteractions(**common_params).to(device)
elif MODEL_TYPE == "ModelB":
    model = ModelB_WithInteractions(**common_params).to(device)
else:
    model = PTCNet(**common_params).to(device)

# Loss & Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=3, verbose=True
)

print(model)

In [ ]:
# --- Training Utils ---

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total_samples = 0
    
    for batch in loader:
        # Unpack batch (handling the 4-tuple from collate)
        X = batch[0].to(device)
        y = batch[1].to(device).float().view(-1) # (B,)
        # feat_mask = batch[3].to(device) # Optional: use if model supports it
        
        optimizer.zero_grad()
        
        # Forward
        # Note: We can pass feature_mask=feat_mask if we want to be strict about padding
        logits = model(X) # (B,)
        
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * X.size(0)
        
        # Accuracy
        preds = (torch.sigmoid(logits) >= 0.5).float()
        correct += (preds == y).sum().item()
        total_samples += X.size(0)
        
    return total_loss / total_samples, correct / total_samples

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for batch in loader:
            X = batch[0].to(device)
            y = batch[1].to(device).float().view(-1)
            
            logits = model(X)
            loss = criterion(logits, y)
            
            total_loss += loss.item() * X.size(0)
            preds = (torch.sigmoid(logits) >= 0.5).float()
            correct += (preds == y).sum().item()
            total_samples += X.size(0)
            
    return total_loss / total_samples, correct / total_samples

In [ ]:
# --- Main Training Loop ---
num_epochs = 20
best_val_loss = float("inf")
os.makedirs("checkpoints", exist_ok=True)

print("Starting training...")

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    
    scheduler.step(val_loss)
    
    print(f"Epoch {epoch+1:02d} | train_loss={train_loss:.4f}, train_acc={train_acc:.4f}, val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "checkpoints/ptcnet_best.pth")
        print("  --> Saved best model")

In [ ]:
# --- Final Test Evaluation ---
if os.path.exists("checkpoints/ptcnet_best.pth"):
    print("Loading best model for testing...")
    model.load_state_dict(torch.load("checkpoints/ptcnet_best.pth", map_location=device))

test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

### Model Summary
- **Input**: (Batch, Features, Time) - Time is variable per batch.
- **Architecture**: PTCNet (Residual 1D Conv) compresses the temporal dimension.
- **Output**: Single scalar logit per sample (Binary Classification).